# 1. Setting up the environment

## 1.1 Load Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns  # For improved visualization aesthetics
from adjustText import adjust_text  # This library helps in adjusting text
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score


import warnings
warnings.filterwarnings('ignore')

## 1.2 Load Data

In [2]:
train_data_raw = pd.read_csv("data/FinSalesPriceData_train.csv")
test_data_raw = pd.read_csv("data/FinSalesPriceData_test.csv")
competitor_data_raw = pd.read_csv("data/CompetitorPriceData.csv")

In [3]:
train_data_raw.head()

,calendar_day,article_id,article_desc,category,subcategory,segment,brand,brandtype,sell_price,promo_price,promo_sales,promo_units,gross_profit,scanback,sales_amount,sales_units,cnt_site_art_ranged,cnt_site_art_ranged_pstv_soh,tot_soh_ranged_sites,gst_flag
0,2022-02-07,196544,Skin Control Pimple Patch Micro Dart 9pk,Skin & Sun Care,Skincare Face,Other,SKIN CONTROL,National Brand,13.00,13.0,13.0,1,86.550,0.0,190.7080,15,176,171,1444.0,Y
1,2022-02-07,103515,Spascriptions Superfoods Masks 3x 50ml,Skin & Sun Care,Skincare Face,Masks,SPASCRIPTIONS,National Brand,20.00,NaN,NaN,0,28.719,0.0,59.7999,3,176,175,1907.0,Y
2,2022-02-07,103517,Spascriptions Retinol Facial Serum,Skin & Sun Care,Skincare Face,Masks,SPASCRIPTIONS,National Brand,18.00,NaN,NaN,0,17.158,0.0,35.8198,2,176,174,844.0,Y
3,2022-02-07,103518,Spascriptions Collagen Facial Serum,Skin & Sun Care,Skincare Face,Masks,SPASCRIPTIONS,National Brand,18.00,NaN,NaN,0,42.926,0.0,89.6396,5,176,174,938.0,Y
4,2022-02-07,103520,7th Heaven Blackhead Stardust Face Mask,Skin & Sun Care,Skincare Face,7Th Heaven,7TH HEAVEN,National Brand,6.95,NaN,NaN,0,16.919,0.0,41.5209,6,176,173,1960.0,Y


## 1.3 Make a copy of the data

In [4]:
# make a copy of the data
data_train = train_data_raw.copy()
data_test = test_data_raw.copy()
data_c = competitor_data_raw.copy()

# 2. Data Preprocessing

## 2.1 Split the train data for train and validation sets

In [5]:
data_train_idx, data_val_idx = train_test_split(data_train.index, test_size=0.2, random_state=3600)
data_train = train_data_raw.loc[data_train_idx]
data_val = train_data_raw.loc[data_val_idx]

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)


(1000013, 20)
(250004, 20)
(605777, 20)


## 2.2 Missing Value Management

In [6]:
# function that process the selling price data
def missing_sell_price(data):
    # Calculate NA sell price using sales amount / sales units
    data['sell_price'] = data['sell_price'].fillna(data['sales_amount']/data['sales_units'])
    # Use previous sell price of no sales amount
    data['sell_price'] = data['sell_price'].fillna(method='ffill')
    return data

# function that process missing promo price and promo sales
def missing_promo(data):
    for idx in data.index:
        if pd.isna(data['promo_price'][idx]) and pd.notna(data['promo_sales'][idx]):
            data['promo_sales'][idx] = np.nan
    return data

In [7]:
data_train = missing_sell_price(data_train)
data_train = missing_promo(data_train)

data_val = missing_sell_price(data_val)
data_val = missing_promo(data_val)

data_test = missing_sell_price(data_test)
data_test = missing_promo(data_test)

In [8]:
print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(1000013, 20)
(250004, 20)
(605777, 20)


## 2.3 Inconsistencies and Anomalies

### 2.3.1 Negative Sell Price, Promotion Price, Promotional Sales, and Scanback

In [9]:
# function that drop negative promotion negative, negative sell price
def drop_neg_invalid_rows(data):
    data = data.drop(data[data['promo_price'] < 0].index)
    data = data.drop(data[data['promo_sales'] < 0].index)
    data = data.drop(data[data['scanback'] < 0].index)
    data = data.drop(data[data['sell_price'] < 0].index)
    return data

data_train = drop_neg_invalid_rows(data_train)
data_val = drop_neg_invalid_rows(data_val)
data_test = drop_neg_invalid_rows(data_test)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(997831, 20)
(249465, 20)
(604344, 20)


### 2.3.2 Availability Issues

In [10]:
# function that drop rows with availability issues
def rm_availability_issue(data):
    condition = data[(data['cnt_site_art_ranged'] > 0) & (data['tot_soh_ranged_sites'] == 0)]
    data = data.drop(condition.index)
    return data

data_train = rm_availability_issue(data_train)
data_val = rm_availability_issue(data_val)
data_test = rm_availability_issue(data_test)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(972425, 20)
(243254, 20)
(596838, 20)


### 2.3.3 Invalid Sales (Sales Units and Promotional Units)

In [11]:
# function that drop rows where promo units or sales units are more than stock
def drop_sell_more_than_stock(data):
    data = data.drop(data[data['sales_units'] > data['tot_soh_ranged_sites']].index)
    data = data.drop(data[data['promo_units'] > data['tot_soh_ranged_sites']].index)
    return data

data_train = drop_sell_more_than_stock(data_train)
data_val = drop_sell_more_than_stock(data_val)
data_test = drop_sell_more_than_stock(data_test)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(919962, 20)
(230080, 20)
(562224, 20)


### 2.3.4 Duplicates

In [12]:
# function that remove duplicates
def remove_duplicates(data):
    duplicate = data.duplicated()
    data = data.drop(data[duplicate].index)
    return data

data_train = remove_duplicates(data_train)
data_val = remove_duplicates(data_val)
data_test = remove_duplicates(data_test)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(919962, 20)
(230080, 20)
(562224, 20)


### 2.3.5 Promo Price higher and equal to Sell Price

In [13]:
def remove_invalid_promo(data):
    data = data.drop(data[(data['promo_price'] >= data['sell_price'])].index)
    return data

data_train = remove_invalid_promo(data_train)
data_val = remove_invalid_promo(data_val)
data_test = remove_invalid_promo(data_test)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

(919146, 20)
(229874, 20)
(561534, 20)


### 2.3.5 Drop Inventory Features
- This is because we won't know for certain what the inventory levels will be in the future.

In [14]:
data_train = data_train.drop(['cnt_site_art_ranged', 'cnt_site_art_ranged_pstv_soh', 'tot_soh_ranged_sites'], axis=1)
data_val = data_val.drop(['cnt_site_art_ranged', 'cnt_site_art_ranged_pstv_soh', 'tot_soh_ranged_sites'], axis=1)
data_test = data_test.drop(['cnt_site_art_ranged', 'cnt_site_art_ranged_pstv_soh', 'tot_soh_ranged_sites'], axis=1)

## 2.4 Data Types Transformation
1. Convert `calendar_day` to datetime
2. Create dummy variables for categorical features
    - `article_desc` does not have a clear categorisation, so we will not create dummy for it

In [15]:
def cat_to_dum(data, var_name):
    category_dummies = pd.get_dummies(data[var_name], prefix=var_name)
    data = pd.concat([data, category_dummies], axis=1)
    data = data.drop(var_name, axis=1)
    return data
    
def data_type_transform(data):
    # find the categorical columns
    cat_columns = data.select_dtypes(include=['object']).columns
    
    # transform calendar_day to datetime object
    data['calendar_day'] = pd.to_datetime(data['calendar_day'])
    
    # transform category to dummy
    data = cat_to_dum(data, 'category')
    
    # transform sub_category to dummy
    data = cat_to_dum(data, 'subcategory')
    
    # transform segment to dummy
    data = cat_to_dum(data, 'segment')
    
    # transform brand to dummy
    data = cat_to_dum(data, 'brand')
    
    # transform brandtype to dummy
    data = cat_to_dum(data, 'brandtype')
    
    # transform gst to dummy
    data = cat_to_dum(data, 'gst_flag')

    return data

In [16]:
data_train_with_dummy = data_train.copy()
data_train_with_dummy = data_type_transform(data_train_with_dummy)

data_val_with_dummy = data_val.copy()
data_test_with_dummy = data_test.copy()

data_val_with_dummy = data_type_transform(data_val_with_dummy)
data_test_with_dummy = data_type_transform(data_test_with_dummy)

In [17]:
print(data_train_with_dummy.shape)
print(data_val_with_dummy.shape)
print(data_test_with_dummy.shape)

(919146, 413)
(229874, 413)
(561534, 413)


## 2.5 Additional Variables
1. Revised profits:
    `net_proft` = `gross_profit` + `scanback`

2. Cost of purchasing:
    2.1 if GST is present: `sales_amount_exclude_gst` = `sales_amount` / 1.1
    2.2 if GST is absent: `sales_amount_exclude_gst` = `sales_amount`
    `cost_of_purchasing` = `sales_amount_exclude_gst` - `net_profit`

3. Sales units per dollar decrease to see the price elasticity of the article:
    `sales_units_per_dollar` = `promo_units` / (`selling_price` - `promo_price`)

4. Day of Week
    `day_of_week` = `date`.dt.dayofweek

5. Whether competitor has a lower selling or shelves price

6. Whether the article is on promotion (0 for no, 1 for yes)


In [18]:
def new_var(data):
    # Calculate revised net profit
    data['net_profit'] = data['gross_profit'] + data['scanback']
    
    # Calculate sales amount excluding GST using vectorized operation
    data['sales_amount_exclude_gst'] = data['sales_amount'] / (1.1 if 'gst_flag_Y' in data.columns and data['gst_flag_Y'].all() else 1)
    
    # Calculate cost of purchasing
    data['cost_of_purchasing'] = data['sales_amount_exclude_gst'] - data['net_profit']
    
    # Calculate sales units per dollar decrease due to promotions
    data['marginal_sales_per_dollar_decrease'] = data['promo_units'] / (data['sell_price'] - data['promo_price'])
    
    # Extract day of week from 'calendar_day'
    data['day_of_week'] = data['calendar_day'].dt.dayofweek
    ## treat day_of_week as categorical
    data = cat_to_dum(data, 'day_of_week')
    
    # Identify whether the article is on promotion
    data['on_promo'] = data['promo_price'].apply(lambda x: 1 if x > 0 else 0)
    ## change on_promo to bool
    data['on_promo'] = data['on_promo'].astype(bool)
    
    return data

data_train_with_dummy = new_var(data_train_with_dummy)
data_val_with_dummy = new_var(data_val_with_dummy)
data_test_with_dummy = new_var(data_test_with_dummy)

In [19]:
print(data_train_with_dummy.shape)
print(data_val_with_dummy.shape)
print(data_test_with_dummy.shape)

(919146, 425)
(229874, 425)
(561534, 425)


## 2.6 Standardisation

### 2.6.1 Split into normal and promotional data

In [20]:
def normal_promo_sets(data):
    return data[data["promo_price"].isna()], data[data["promo_price"].notna()]

# Split data into Normal Sales and Promo Sales
train_normal, train_promo = normal_promo_sets(data_train_with_dummy)
test_normal, test_promo = normal_promo_sets(data_test_with_dummy)
val_normal, val_promo = normal_promo_sets(data_val_with_dummy)

# check shape
print(train_normal.shape)
print(train_promo.shape)
print(test_normal.shape)
print(test_promo.shape)
print(val_normal.shape)
print(val_promo.shape)

(733589, 425)
(185557, 425)
(441888, 425)
(119646, 425)
(183182, 425)
(46692, 425)


### 2.6.2 Standardise the data

In [21]:
# check numerical columns
train_normal.select_dtypes(include=['int', 'float']).columns

# colns that have na values for normal sales
print(train_normal.columns[train_normal.isna().any()])

# colns that have na values for promo sales
print(train_promo.columns[train_promo.isna().any()])
print(train_promo.columns[train_promo.isin([np.nan, np.inf, -np.inf]).any()])
print(train_promo['marginal_sales_per_dollar_decrease'].value_counts())

Index(['promo_price', 'promo_sales', 'marginal_sales_per_dollar_decrease'], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
marginal_sales_per_dollar_decrease
2.000000      573
4.000000      503
1.000000      483
0.666667      421
1.333333      396
             ... 
56.787330       1
22.096317       1
20.487805       1
0.804598        1
489.189189      1
Name: count, Length: 57978, dtype: int64


In [22]:
def prepare_for_std(data, promo=False):
    # rm the columns that are misclassified as numerical
    data['article_id'] = data_train_with_dummy['article_id'].astype(str)
    
    # consider two cases: normal sales and promo sales
    ## for normal sales drop promo columns
    if promo == False:
        data = data.drop(['promo_price', 'promo_sales', 'marginal_sales_per_dollar_decrease'], axis=1)
    return data

train_normal = prepare_for_std(train_normal)
train_promo = prepare_for_std(train_promo, promo=True)
val_normal = prepare_for_std(val_normal)
val_promo = prepare_for_std(val_promo, promo=True)
test_normal = prepare_for_std(test_normal)
test_promo = prepare_for_std(test_promo, promo=True)

# check shape
print(train_normal.shape)
print(train_promo.shape)
print(val_normal.shape)
print(val_promo.shape)
print(test_normal.shape)
print(test_promo.shape)

(733589, 422)
(185557, 425)
(183182, 422)
(46692, 425)
(441888, 422)
(119646, 425)


In [23]:
train_normal_unstd = train_normal.copy()
train_promo_unstd = train_promo.copy()
val_normal_unstd = val_normal.copy()
val_promo_unstd = val_promo.copy()
test_normal_unstd = test_normal.copy()
test_promo_unstd = test_promo.copy()
# check shape
print(train_normal_unstd.shape)
print(train_promo_unstd.shape)
print(val_normal_unstd.shape)
print(val_promo_unstd.shape)
print(test_normal_unstd.shape)
print(test_promo_unstd.shape)

(733589, 422)
(185557, 425)
(183182, 422)
(46692, 425)
(441888, 422)
(119646, 425)


In [24]:
# standardise all numerical columns
def standardise(data):    
    # standardise all numerical columns
    scaler = StandardScaler()
    data[data.select_dtypes(include=['int', 'float']).columns] = scaler.fit_transform(data.select_dtypes(include=['int', 'float']))
    return data

train_normal = standardise(train_normal)
train_promo = standardise(train_promo)
val_normal = standardise(val_normal)
val_promo = standardise(val_promo)
test_normal = standardise(test_normal)
test_promo = standardise(test_promo)

# check shape
print(train_normal.shape)
print(train_promo.shape)
print(val_normal.shape)
print(val_promo.shape)
print(test_normal.shape)
print(test_promo.shape)

(733589, 422)
(185557, 425)
(183182, 422)
(46692, 425)
(441888, 422)
(119646, 425)


## 2.7 Split data into x features and y target

In [25]:
def y_x_split(data):
    return data["sales_units"], data.drop("sales_units", axis =1)

# Split data into x and y sets
y_train_normal, x_train_normal = y_x_split(train_normal)
y_train_promo, x_train_promo = y_x_split(train_promo)

y_test_normal, x_test_normal = y_x_split(test_normal)
y_test_promo, x_test_promo = y_x_split(test_promo)

y_val_normal, x_val_normal = y_x_split(val_normal)
y_val_promo, x_val_promo = y_x_split(val_promo)

# check shape
print(y_train_normal.shape)
print(x_train_normal.shape)
print(y_train_promo.shape)
print(x_train_promo.shape)

print(y_test_normal.shape)
print(x_test_normal.shape)
print(y_test_promo.shape)
print(x_test_promo.shape)

print(y_val_normal.shape)
print(x_val_normal.shape)
print(y_val_promo.shape)
print(x_val_promo.shape)

(733589,)
(733589, 421)
(185557,)
(185557, 424)
(441888,)
(441888, 421)
(119646,)
(119646, 424)
(183182,)
(183182, 421)
(46692,)
(46692, 424)


In [26]:
x_test_promo.head()

,calendar_day,article_id,article_desc,sell_price,promo_price,promo_sales,promo_units,gross_profit,scanback,sales_amount,...,cost_of_purchasing,marginal_sales_per_dollar_decrease,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,on_promo
1,2023-05-26,NaN,Oral B VitalityFlossAction ToothBrush,1.963083,1.292222,-0.413041,-0.752682,0.400058,-0.376137,-0.024675,...,-0.011325,-0.509476,False,False,False,False,True,False,False,True
4,2023-05-26,103520,COLGATE ADV TEETH WHITENG TPASTE 200G,-0.428994,-1.051363,-0.434208,-0.694642,0.478994,-0.383670,-0.059429,...,-0.068998,-0.500297,False,False,False,False,True,False,False,True
5,2023-05-26,103521,Colgate Adv Wht TrtrCntrl Tpaste 200g,-0.428994,-0.585949,0.514717,2.512056,-0.708254,1.206133,0.523631,...,0.391303,0.353534,False,False,False,False,True,False,False,True
7,2023-05-26,NaN,COLGATE MAX FRESH COOL MINT TPASTE 200G,-0.428994,-0.633559,-0.396393,-0.622093,0.538462,-0.355779,-0.027621,...,-0.057375,-0.475315,False,False,False,False,True,False,False,True
21,2023-05-26,892758,Colgate Toothbrush Zigzag Flex Med3pk,-0.548598,-0.663680,0.332281,2.395977,-0.585127,0.517941,0.281282,...,0.305521,0.572919,False,False,False,False,True,False,False,True


# 4. Feature Selection and Model Building
- using pca to reduce dimention of the data (use loadings to identify the most important features)
- then use the relatively important features to build the model, where each model will inform about the feature importance

## 4.1 PCA

In [27]:
def pca(x_data):
    ## We set 0.95 meaning that we want to keep 95% of the variance -> i.e. pca.explained_variance_ratio_ is 0.95
    pca = PCA(n_components=0.95, random_state=3600)

    ## selecting numerical columns for PCA
    data_num = x_data.select_dtypes(include=['int', 'float', 'bool'])

    ## fit and transform the data to get the principal components
    data_pca = pca.fit_transform(data_num)
    
    ## check the number of principal components that can explain 95% of the variance
    print(f"Number of principal components: {pca.n_components_}")
    
    ## check the loadings which will be used to interpret the principal components
    loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(pca.components_.shape[0])], index=data_num.columns)

    ## use CI to determine the threshold
    mean_loadings = loadings.abs().mean(axis=1).sort_values(ascending=False)
    std_loadings = loadings.abs().std(axis=1).sort_values(ascending=False)

    std_error = std_loadings / np.sqrt(loadings.shape[1])

    ## calculate the 95% CI
    ci_lower = mean_loadings - 1.96 * std_error
    ci_upper = mean_loadings + 1.96 * std_error

    ## we will use a restrictive percentile (0.75) to determine the threshold
    threshold = ci_lower.quantile(0.75)

    ## select the variables that are significant
    significant_vars = mean_loadings[mean_loadings > threshold].index

    return significant_vars

def dimension_red(x_data):
    significant_features = pca(x_data)
    x_data2 = x_data.copy()
    x_data2 = x_data2[significant_features]
    return x_data2
    
x_train_normal_pca = dimension_red(x_train_normal)
x_train_promo_pca = dimension_red(x_train_promo)

# validation and test set should contain the same columns as the training set
x_val_normal_pca = x_val_normal[x_train_normal_pca.columns]
x_val_promo_pca = x_val_promo[x_train_promo_pca.columns]

x_test_normal_pca = x_test_normal[x_train_normal_pca.columns]
x_test_promo_pca = x_test_promo[x_train_promo_pca.columns]

print(x_train_normal_pca.shape)
print(x_train_promo_pca.shape)
print(x_val_normal_pca.shape)
print(x_val_promo_pca.shape)
print(x_test_normal_pca.shape)
print(x_test_promo_pca.shape)

Number of principal components: 77
Number of principal components: 50
(733589, 172)
(185557, 145)
(183182, 172)
(46692, 145)
(441888, 172)
(119646, 145)


## 4.1.1 Visualise the explained variance ratio

In [28]:
def pca_biplot(x_data):
    pca = PCA(n_components=2, random_state=3600)
    data_num = x_data.select_dtypes(include=['int', 'float', 'bool'])
    principalComponents = pca.fit_transform(data_num)
    principalDf = pd.DataFrame(data=principalComponents, columns=['PC1', 'PC2'])

    plt.figure(figsize=(12, 10))
    
    # Using a contrasting color 'magenta' for the scatterplot
    sns.scatterplot(x='PC1', y='PC2', data=principalDf, alpha=0.5, color='blue')

    texts = []
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

    for i, col in enumerate(data_num.columns):
        plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='red', alpha=0.9, linewidth=2)
        text = plt.text(loadings[i, 0] * 1.1, loadings[i, 1] * 1.1, col, color='black', ha='center', va='center', fontsize=12,
                        bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
        texts.append(text)

    # Using adjustText to automatically adjust label positions
    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='black'))

    plt.xlabel('Principal Component 1 (PC1)')
    plt.ylabel('Principal Component 2 (PC2)')
    plt.title('Enhanced PCA Biplot with Contrasting Colors')
    plt.grid(True)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.show()
    
# pca_biplot(x_test_promo)

## 4.2 Decision Tree Regressor

In [29]:
x_train_normal_dt = x_train_normal_pca[x_train_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_train_promo_dt = x_train_promo_pca[x_train_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]

x_val_normal_dt = x_val_normal_pca[x_val_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_val_promo_dt = x_val_promo_pca[x_val_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]

x_test_normal_dt = x_test_normal_pca[x_test_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_test_promo_dt = x_test_promo_pca[x_test_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]

In [30]:
# evaluate the rf model
def evaluate_dt(X_train, y_train, X_val, y_val, X_test, y_test):

    X_train_dt = X_train
    X_val_dt = X_val
    X_test_dt = X_test
    
    ## hyperparameter tuning
    best_rmse = float('inf')
    best_max_depth = 0
    for i in np.arange(1, 100, 10):
        print(f"Hyperparameter tuning: max_depth = {i}")
        dt = DecisionTreeRegressor(max_depth=i, random_state=3600)
        dt.fit(X_train_dt, y_train)
        y_val_pred = dt.predict(X_val_dt)
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_max_depth = i
    
    ## best model
    best_dt = DecisionTreeRegressor(max_depth=best_max_depth, random_state=3600)
    best_dt.fit(X_train_dt, y_train)
    
    ## evaluate on test set
    y_test_pred = best_dt.predict(X_test_dt)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return best_dt, best_rmse, test_rmse

dt_normal, val_rmse_normal_dt, test_rmse_normal_dt = evaluate_dt(x_train_normal_dt, y_train_normal, x_val_normal_dt, y_val_normal, x_test_normal_dt, y_test_normal)
dt_promo, val_rmse_promo_dt, test_rmse_promo_dt = evaluate_dt(x_train_promo_dt, y_train_promo, x_val_promo_dt, y_val_promo, x_test_promo_dt, y_test_promo)

Hyperparameter tuning: max_depth = 1
Hyperparameter tuning: max_depth = 11
Hyperparameter tuning: max_depth = 21
Hyperparameter tuning: max_depth = 31
Hyperparameter tuning: max_depth = 41
Hyperparameter tuning: max_depth = 51
Hyperparameter tuning: max_depth = 61
Hyperparameter tuning: max_depth = 71
Hyperparameter tuning: max_depth = 81
Hyperparameter tuning: max_depth = 91
Hyperparameter tuning: max_depth = 1
Hyperparameter tuning: max_depth = 11
Hyperparameter tuning: max_depth = 21
Hyperparameter tuning: max_depth = 31
Hyperparameter tuning: max_depth = 41
Hyperparameter tuning: max_depth = 51
Hyperparameter tuning: max_depth = 61
Hyperparameter tuning: max_depth = 71
Hyperparameter tuning: max_depth = 81
Hyperparameter tuning: max_depth = 91


In [31]:
# important features
important_features_normal = pd.DataFrame({'Feature': x_train_normal_dt.columns, 'Importance': dt_normal.feature_importances_})
important_features_normal = important_features_normal.sort_values(by='Importance', ascending=False)

important_features_promo = pd.DataFrame({'Feature': x_train_promo_dt.columns, 'Importance': dt_promo.feature_importances_})
important_features_promo = important_features_promo.sort_values(by='Importance', ascending=False)

# format the output in a table
output = pd.DataFrame({'Model': ['Decision Tree for Normal Sales', 'Decision Tree for Promo Sales'],
                       'Best Model Parameters': [dt_normal.max_depth, dt_promo.max_depth],
                       'Top 10 Important Features': [important_features_normal['Feature'].head(10).values, important_features_promo['Feature'].head(10).values],
                       'Validation RMSE': [val_rmse_normal_dt, val_rmse_promo_dt],
                       'Test RMSE': [test_rmse_normal_dt, test_rmse_promo_dt]})
                      
pd.set_option('display.max_colwidth', None)
output

,Model,Best Model Parameters,Top 10 Important Features,Validation RMSE,Test RMSE
0,Decision Tree for Normal Sales,71,"[sales_amount, sell_price, cost_of_purchasing, sales_amount_exclude_gst, gross_profit, net_profit, brandtype_Private Label, category_Skin & Sun Care, subcategory_Soaps & Wash, segment_Kitchen]",0.040936,0.092437
1,Decision Tree for Promo Sales,31,"[promo_units, cost_of_purchasing, brandtype_National Brand, sales_amount_exclude_gst, sell_price, sales_amount, marginal_sales_per_dollar_decrease, gross_profit, segment_Huggies Essentials, brand_REXONA]",0.115428,0.114445


## 4.3 Random Forest Regressor

In [32]:
x_train_normal_rf = x_train_normal_pca[x_train_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_train_promo_rf = x_train_promo_pca[x_train_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]

x_test_normal_rf = x_test_normal_pca[x_test_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_test_promo_rf = x_test_promo_pca[x_test_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]

x_val_normal_rf = x_val_normal_pca[x_val_normal_pca.select_dtypes(include=['int', 'float', 'bool']).columns]
x_val_promo_rf = x_val_promo_pca[x_val_promo_pca.select_dtypes(include=['int', 'float', 'bool']).columns]


In [33]:
# evaluate the rf model
def evaluate_rf(X_train, y_train, X_val, y_val, X_test, y_test, std = True):

    X_train_rf = X_train
    X_val_rf = X_val
    X_test_rf = X_test
    
    ## hyperparameter tuning
    best_rmse = float('inf')
    best_n_estimators = 0
    for i in np.arange(1, 10, 2):
        print(f"Hyperparameter tuning: n_estimators = {i}")
        rf = RandomForestRegressor(n_estimators=i, random_state=3600)
        rf.fit(X_train_rf, y_train)
        y_val_pred = rf.predict(X_val_rf)
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_n_estimators = i
    
    ## best model
    best_rf = RandomForestRegressor(n_estimators=best_n_estimators, random_state=3600)
    best_rf.fit(X_train_rf, y_train)
    
    ## evaluate on test set
    y_test_pred = best_rf.predict(X_test_rf)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return best_rf, val_rmse, test_rmse


rf_normal, val_rmse_normal_rf, test_rmse_normal_rf = evaluate_rf(x_train_normal_rf, y_train_normal, x_val_normal_rf, y_val_normal, x_test_normal_rf, y_test_normal)
rf_promo, val_rmse_promo_rf, test_rmse_promo_rf = evaluate_rf(x_train_promo_rf, y_train_promo, x_val_promo_rf, y_val_promo, x_test_promo_rf, y_test_promo)


Hyperparameter tuning: n_estimators = 1
Hyperparameter tuning: n_estimators = 3
Hyperparameter tuning: n_estimators = 5
Hyperparameter tuning: n_estimators = 7
Hyperparameter tuning: n_estimators = 9
Hyperparameter tuning: n_estimators = 1
Hyperparameter tuning: n_estimators = 3
Hyperparameter tuning: n_estimators = 5
Hyperparameter tuning: n_estimators = 7
Hyperparameter tuning: n_estimators = 9


In [34]:
# important features
important_features_normal = pd.DataFrame({'Feature': x_train_normal_rf.columns, 'Importance': rf_normal.feature_importances_})
important_features_normal = important_features_normal.sort_values(by='Importance', ascending=False)

important_features_promo = pd.DataFrame({'Feature': x_train_promo_rf.columns, 'Importance': rf_promo.feature_importances_})
important_features_promo = important_features_promo.sort_values(by='Importance', ascending=False)

# format the output in a table
output = pd.DataFrame({'Model': ['RF for Normal Sales', 'RF for Promo Sales'],
                       'Best Model Parameters': [rf_normal.n_estimators, rf_promo.n_estimators],
                       'Top 10 Important Features': [important_features_normal['Feature'].head(10).values, important_features_promo['Feature'].head(10).values],
                       'Validation RMSE': [val_rmse_normal_rf, val_rmse_promo_rf],
                       'Test RMSE': [test_rmse_normal_rf, test_rmse_promo_rf]})
                      
pd.set_option('display.max_colwidth', None)
output

,Model,Best Model Parameters,Top 10 Important Features,Validation RMSE,Test RMSE
0,RF for Normal Sales,9,"[sales_amount, sales_amount_exclude_gst, sell_price, cost_of_purchasing, net_profit, gross_profit, brandtype_Manufacturer Brand, subcategory_Soaps & Wash, segment_Bar Soap, brandtype_National Brand]",0.052532,0.081294
1,RF for Promo Sales,9,"[promo_units, cost_of_purchasing, brandtype_National Brand, sales_amount_exclude_gst, sell_price, sales_amount, promo_price, gross_profit, marginal_sales_per_dollar_decrease, segment_Huggies Essentials]",0.092181,0.103077


# 5. Model Evaluation of the Best Model (Random Forest Regressor)
- Destandardise the error so that it is interpretable

In [35]:
y_val_normal_unstd, x_val_normal_unstd = y_x_split(val_normal_unstd)
y_val_promo_unstd, x_val_promo_unstd = y_x_split(val_promo_unstd)
y_test_normal_unstd, x_test_normal_unstd = y_x_split(test_normal_unstd)
y_test_promo_unstd, x_test_promo_unstd = y_x_split(test_promo_unstd)


rf_normal, val_rmse_normal_rf, test_rmse_normal_rf = evaluate_rf(x_train_normal_rf, y_train_normal, x_val_normal_rf, y_val_normal_unstd, x_test_normal_rf, y_test_normal_unstd)
rf_promo, val_rmse_promo_rf, test_rmse_promo_rf = evaluate_rf(x_train_promo_rf, y_train_promo, x_val_promo_rf, y_val_promo_unstd, x_test_promo_rf, y_test_promo_unstd)

Hyperparameter tuning: n_estimators = 1
Hyperparameter tuning: n_estimators = 3
Hyperparameter tuning: n_estimators = 5
Hyperparameter tuning: n_estimators = 7
Hyperparameter tuning: n_estimators = 9
Hyperparameter tuning: n_estimators = 1
Hyperparameter tuning: n_estimators = 3
Hyperparameter tuning: n_estimators = 5
Hyperparameter tuning: n_estimators = 7
Hyperparameter tuning: n_estimators = 9


In [36]:
# important features
important_features_normal = pd.DataFrame({'Feature': x_train_normal_rf.columns, 'Importance': rf_normal.feature_importances_})
important_features_normal = important_features_normal.sort_values(by='Importance', ascending=False)

important_features_promo = pd.DataFrame({'Feature': x_train_promo_rf.columns, 'Importance': rf_promo.feature_importances_})
important_features_promo = important_features_promo.sort_values(by='Importance', ascending=False)

# format the output in a table
output = pd.DataFrame({'Model': ['RF for Normal Sales', 'RF for Promo Sales'],
                       'Best Model Parameters': [rf_normal.n_estimators, rf_promo.n_estimators],
                       'Top 10 Important Features': [important_features_normal['Feature'].head(10).values, important_features_promo['Feature'].head(10).values],
                       'Validation RMSE': [val_rmse_normal_rf, val_rmse_promo_rf],
                       'Test RMSE': [test_rmse_normal_rf, test_rmse_promo_rf]})
                      
pd.set_option('display.max_colwidth', None)
output

,Model,Best Model Parameters,Top 10 Important Features,Validation RMSE,Test RMSE
0,RF for Normal Sales,3,"[sales_amount, sales_amount_exclude_gst, sell_price, cost_of_purchasing, net_profit, brandtype_Manufacturer Brand, gross_profit, day_of_week_5, segment_Bar Soap, brandtype_National Brand]",61.975598,55.292358
1,RF for Promo Sales,9,"[promo_units, cost_of_purchasing, brandtype_National Brand, sales_amount_exclude_gst, sell_price, sales_amount, promo_price, gross_profit, marginal_sales_per_dollar_decrease, segment_Huggies Essentials]",94.587343,88.887927


# 6. Bundle Creation
- Promo is more prone to change the sales units than Normal Sales, hence even though promo sale RF model has a lower RMSE, we will use the Promo Sales model to create the bundles

In [37]:
pd.set_option('display.max_colwidth', None)
all_important_features = rf_promo.feature_importances_

# get the features variables with the importance
pd.set_option('display.max_rows', None)
important_features = pd.DataFrame({'Feature': x_train_promo_rf.columns, 'Importance': all_important_features})
important_features

# see only the subcategory
pd.set_option('display.max_rows', None)
subcat = important_features[important_features['Feature'].str.contains('subcategory')]

# rank by importance
pd.set_option('display.max_rows', None)
subcat = subcat.sort_values(by='Importance', ascending=False)
subcat

,Feature,Importance
11,subcategory_Cleaning Products,7.254915e-04
65,subcategory_Baby Wipes,2.786195e-04
10,subcategory_Skincare Body,2.773451e-05
15,subcategory_Laundry Detergent,1.468540e-05
58,subcategory_Deodorants,1.098158e-05
39,subcategory_Prewash & Conditio,7.920214e-06
7,subcategory_Disposable Nappies,3.196743e-06
50,subcategory_Oral Care,3.115163e-06
17,subcategory_Baby Toiletries,2.854176e-06
8,subcategory_Baby Food,1.927799e-06


## 6.1 Bundle Combination (Subcategory)
- Using the promo sales model, extracting the top 3 subcat and making combinations with the rest of the subcats to create bundles

In [38]:
# select the top 3 subcategories based on rf importance as the top sales drivers
top3_subcat = subcat.head(3)

# other subcategories
other_subcat = subcat.tail(subcat.shape[0] - 3)

In [39]:
# combine the top 3 subcategories and other subcategories
bundle_data = pd.DataFrame()
bundle_data['Top Subcategory'] = np.repeat(top3_subcat['Feature'], len(other_subcat))
bundle_data['Other Subcategory'] = np.tile(other_subcat['Feature'], len(top3_subcat))

bundle_data = bundle_data.reset_index(drop=True)
print(bundle_data.shape)
bundle_data.head()

(48, 2)


,Top Subcategory,Other Subcategory
0,subcategory_Cleaning Products,subcategory_Laundry Detergent
1,subcategory_Cleaning Products,subcategory_Deodorants
2,subcategory_Cleaning Products,subcategory_Prewash & Conditio
3,subcategory_Cleaning Products,subcategory_Disposable Nappies
4,subcategory_Cleaning Products,subcategory_Oral Care


## 6.2 Generate Bundles ID
- creating unique article id for each bundle, treating as a new article

In [40]:
# Create new article id for the bundle data
def create_bundle_id(bundle_data, original_data):
    # generate a new article id for the bundle data that's not in the original data
    existing_id = original_data['article_id'].unique()
    bundle_data['bundle_id'] = [0]*bundle_data.shape[0]
    
    # create a new article id for the bundle data
    np.random.seed(3600)
    for idx in bundle_data.index:
        new_id = np.random.randint(100000, 999999)
        while new_id in existing_id:
            new_id = np.random.randint(100000, 999999)
            existing_id.append(new_id)
        bundle_data['bundle_id'][idx] = new_id
            
    return bundle_data

bundle_data = create_bundle_id(bundle_data, data_train_with_dummy)
bundle_data.head()

,Top Subcategory,Other Subcategory,bundle_id
0,subcategory_Cleaning Products,subcategory_Laundry Detergent,430030
1,subcategory_Cleaning Products,subcategory_Deodorants,505208
2,subcategory_Cleaning Products,subcategory_Prewash & Conditio,220109
3,subcategory_Cleaning Products,subcategory_Disposable Nappies,392445
4,subcategory_Cleaning Products,subcategory_Oral Care,285107


## 6.3 Select the specific articles for the bundles
- Top 3 subcategories - 3 articles that has the highest median sales units under the subcategory
- Rest of the subcategories - choose the least median sales units article

In [41]:
def select_bundle_article(bundle_data, original_data):
    for idx in bundle_data.index:
        top_cat = bundle_data['Top Subcategory'][idx]
        other_cat = bundle_data['Other Subcategory'][idx]
        
        # compute the median sales units for the top category
        top_cat = original_data[original_data[top_cat] == 1]
        # for each article in the top category, compute the median sales units
        top_cat_median = top_cat.groupby('article_id')['sales_units'].median()
        # rank from highest to lowest
        top_cat_median = top_cat_median.sort_values(ascending=False)
        # make sure the top article contain promo price
        for article_id in top_cat_median.index:
            # check if the article's promo price is all na
            if original_data[original_data['article_id'] == article_id]['promo_price'].isna().all():
                continue
            else:
                top_article_id = article_id
                break
        # add the article id to the bundle data
        bundle_data.loc[idx, 'Top Article ID'] = top_article_id
        # add description
        bundle_data.loc[idx, 'Top Article Description'] = original_data[original_data['article_id'] == top_article_id]['article_desc'].values[0]
        
        # compute the median sales units for the other category
        other_cat = original_data[original_data[other_cat] == 1]
        # for each article in the other category, compute the median sales units
        other_cat_median = other_cat.groupby('article_id')['sales_units'].median()
        # rank from lowest to highest
        other_cat_median = other_cat_median.sort_values(ascending=True)
        # make sure the other article contain promo price
        for article_id in other_cat_median.index:
            # check if the article's promo price is all na
            if original_data[original_data['article_id'] == article_id]['promo_price'].isna().all():
                continue
            # check if the sell units is 0
            elif original_data[original_data['article_id'] == article_id]['sales_units'].median() == 0:
                continue
            else:
                other_article_id = article_id
                break
        # add the article id to the bundle data
        bundle_data.loc[idx, 'Other Article ID'] = other_article_id
        # add description
        bundle_data.loc[idx, 'Other Article Description'] = original_data[original_data['article_id'] == other_article_id]['article_desc'].values[0]
        
    return bundle_data
        
        
bundle_data = select_bundle_article(bundle_data, data_train_with_dummy)
print(bundle_data.shape)

(48, 7)


In [42]:
data_train_with_dummy.shape
x_train_normal_rf.shape

(733589, 172)

In [43]:
for i in bundle_data.index:
    row = bundle_data.loc[i]
    high_id = row['Top Article ID']
    high_id_desc = row['Top Article Description']
    high_id_median_sales_unit = data_train_with_dummy[data_train_with_dummy['article_id'] == high_id]['sales_units'].median()
    low_id = row['Other Article ID']
    low_id_desc = row['Other Article Description']
    low_id_median_sales_unit = data_train_with_dummy[data_train_with_dummy['article_id'] == low_id]['sales_units'].median()
    print(f"High Performing Article: {high_id_desc} with median sales units: {high_id_median_sales_unit}")
    print(f"Low Performing Article: {low_id_desc} with median sales units: {low_id_median_sales_unit}")
    print("=====================END OF THIS BUNDLE================================")

High Performing Article: Strike Disinfectant Wipes Lemon 100pk with median sales units: 476.0
Low Performing Article: Fab Subblime Velvet 15kg with median sales units: 1.0
=====================END OF THIS BUNDLE================================
High Performing Article: Strike Disinfectant Wipes Lemon 100pk with median sales units: 476.0
Low Performing Article: Dove Powder Soft Roll On 50ml with median sales units: 1.0
=====================END OF THIS BUNDLE================================
High Performing Article: Strike Disinfectant Wipes Lemon 100pk with median sales units: 476.0
Low Performing Article: Vanish Gold Pro OxiAction White2.7kg with median sales units: 1.0
=====================END OF THIS BUNDLE================================
High Performing Article: Strike Disinfectant Wipes Lemon 100pk with median sales units: 476.0
Low Performing Article: Hug UDPant Size 3 Crawler  Boy 36 Bulk with median sales units: 8.0
=====================END OF THIS BUNDLE==========================

## 6.4 Create the bundles dataset
- Create features for the bundles dataset (matching the features in the original dataset)
    - in preparation creation of the dummy variables
    - then eases the fitting of the rf model

In [44]:
def add_bundle_dates(bundle_data, original_data):
    # add the dates in which the top article is not on promotion
    bundle_data['Bundle Dates'] = [0]*bundle_data.shape[0]
    total_calendar_days = 0
    for idx in bundle_data.index:
        ## get the top article id non-promo dates
        top_article_id = bundle_data['Top Article ID'][idx]
        top_article = original_data[original_data['article_id'] == top_article_id]
        top_article_non_promo_dates = top_article[top_article['promo_price'].isna()]['calendar_day']
        top_article_non_promo_dates = top_article_non_promo_dates.tolist()
        
        ## get the other article id non-promo dates
        bottom_article_id = bundle_data['Other Article ID'][idx]
        bottom_article = original_data[original_data['article_id'] == bottom_article_id]
        bottom_article_non_promo_dates = bottom_article[bottom_article['promo_price'].isna()]['calendar_day']
        bottom_article_non_promo_dates = bottom_article_non_promo_dates.tolist()
        
        ## get the intersection of the non-promo dates
        non_promo_dates = list(set(top_article_non_promo_dates).intersection(bottom_article_non_promo_dates))
        bundle_data['Bundle Dates'][idx] = non_promo_dates
        total_calendar_days += len(non_promo_dates)
    return bundle_data, total_calendar_days

bundle_data, total_calendar_days = add_bundle_dates(bundle_data, data_train_with_dummy)
print(bundle_data.shape)

(48, 8)


In [45]:
def combine_dummy_for_bundle(top_cat_df, bottom_cat_df, var_interested):
    top_cat = top_cat_df[top_cat_df.columns[top_cat_df.columns.str.contains(var_interested)]].iloc[0]
    bottom_cat = bottom_cat_df[bottom_cat_df.columns[bottom_cat_df.columns.str.contains(var_interested)]].iloc[0]
    dummy_vars = top_cat | bottom_cat
    return dummy_vars

def dummies_to_bundle_df(top_cat_df, bottom_cat_df, var_interested, num_data):
    combined_dummies = combine_dummy_for_bundle(top_cat_df, bottom_cat_df, var_interested)
    combined_dummies = pd.DataFrame(combined_dummies).T
    # repeat the combined dummies for the number of rows in the bundle data
    combined_dummies = pd.concat([combined_dummies]*num_data, ignore_index=True)
    return combined_dummies

In [46]:
def create_bundle_data(bundle_data, original_data):
    # Create a new dataframe for the bundle data
    bundle_specs_df = pd.DataFrame()
    bundle_data_valid = bundle_data.copy()
    
    for idx in bundle_data.index:
        # # for debugging
        # print(bundle_data.shape)
        # print(f"Currently processing bundle {idx}")
        
        # create a temporary dataframe to store each bundle's data
        temp_df = pd.DataFrame()
        bundle_id = bundle_data['bundle_id'][idx]
        top_article_id = bundle_data['Top Article ID'][idx]
        bottom_article_id = bundle_data['Other Article ID'][idx]
        dates = bundle_data['Bundle Dates'][idx]
        
        top_article_id_info = original_data[original_data['article_id'] == top_article_id]
        bottom_article_id_info = original_data[original_data['article_id'] == bottom_article_id]
        
        # first check if there exist promo price for both articles, if not, get rid of the bundle
        if top_article_id_info['promo_price'].isna().all() or bottom_article_id_info['promo_price'].isna().all():
            bundle_data_valid = bundle_data_valid.drop(idx)
            continue
        
        # then check if gst flag is the same for both articles, if not, hard to determine gst flag for the bundle
        if top_article_id_info['gst_flag_Y'].all() != bottom_article_id_info['gst_flag_Y'].all():
            bundle_data_valid = bundle_data_valid.drop(idx)
            continue
            
        ## add the calendar day for each bundle
        temp_df['calendar_day'] = dates
        
        ## add the bundle id for each bundle
        temp_df['article_id'] = bundle_id
        
        ## add the bundle description for each bundle
        temp_df['article_desc'] = f"{bundle_data['Top Article Description'][idx]} + {bundle_data['Other Article Description'][idx]}"
        
        # add dummy variables for the bundle
        ## 1. add the category and subcategory for each bundle
        cat_dummies = dummies_to_bundle_df(top_article_id_info, bottom_article_id_info, 'category', len(dates))
        temp_df = pd.concat([temp_df, cat_dummies], axis=1)
        
        ## 2. add the segment for each bundle
        segment_dummies = dummies_to_bundle_df(top_article_id_info, bottom_article_id_info, 'segment', len(dates))
        temp_df = pd.concat([temp_df, segment_dummies], axis=1)
        
        ## 3. add the brand and brandtype for each bundle
        brand_dummies = dummies_to_bundle_df(top_article_id_info, bottom_article_id_info, 'brand', len(dates))
        temp_df = pd.concat([temp_df, brand_dummies], axis=1)
        
        ## 4. add the sell price
        ### lowest bundle sell price = top_article median(promo_price) + bottom_article median(promo_price)
        ### highest bundle sell price = top_article median(sell_price) + bottom_article median(promo_price)
        ### take the average of the two as the bundle sell price
        top_article_median_promo_price = top_article_id_info['promo_price'].median()
        top_article_median_sell_price = top_article_id_info['sell_price'].median()
        bottom_article_median_promo_price = bottom_article_id_info['promo_price'].median()
        
        low_threshold = top_article_median_promo_price + bottom_article_median_promo_price
        high_threshold = top_article_median_sell_price + bottom_article_median_promo_price
        
        bundle_price = (low_threshold + high_threshold) / 2
        temp_df['sell_price'] = pd.concat([pd.Series([bundle_price]*len(dates))], ignore_index=True)
        
        ## 5. add the promo price (same as sell price)
        temp_df['promo_price'] = temp_df['sell_price']
        
        ## 6. add the gst flag and convert to dummy
        gst_flag = data_train[data_train['article_id'] == top_article_id]['gst_flag'].values[0]
        temp_df['gst_flag'] = pd.concat([pd.Series([gst_flag]*len(dates))], ignore_index=True)
        temp_df = cat_to_dum(temp_df, 'gst_flag')
        ### all bundle articles contains the same gst flag 'Y', but for data completion, we add both columns
        temp_df['gst_flag_N'] = 0
        
        ## 7. add scanback using the average of the median scanback of the top and bottom articles
        top_article_median_scanback = top_article_id_info['scanback'].median()
        bottom_article_median_scanback = bottom_article_id_info['scanback'].median()
        bundle_scanback = (top_article_median_scanback + bottom_article_median_scanback) / 2
        temp_df['scanback'] = pd.concat([pd.Series([bundle_scanback]*len(dates))], ignore_index=True)

        ## 8. add the median promo sales of the articles and take the average
        promo_sales = top_article_id_info['promo_sales'].median() + bottom_article_id_info['promo_sales'].median()
        promo_sales = promo_sales / 2
        temp_df['promo_sales'] = pd.concat([pd.Series([promo_sales]*len(dates))], ignore_index=True)
        
        ## 9. do the same for promo_units
        promo_units = top_article_id_info['promo_units'].median() + bottom_article_id_info['promo_units'].median()
        promo_units = promo_units / 2
        temp_df['promo_units'] = pd.concat([pd.Series([promo_units]*len(dates))], ignore_index=True)
        
        ## 10. do the same for the gross profit
        gross_profit = top_article_id_info['gross_profit'].median() + bottom_article_id_info['gross_profit'].median()
        gross_profit = gross_profit / 2
        temp_df['gross_profit'] = pd.concat([pd.Series([gross_profit]*len(dates))], ignore_index=True)
        
        ## 11. do the same for the sales_amount
        sales_amount = top_article_id_info['sales_amount'].median() + bottom_article_id_info['sales_amount'].median()
        sales_amount = sales_amount / 2
        temp_df['sales_amount'] = pd.concat([pd.Series([sales_amount]*len(dates))], ignore_index=True)
        
        temp_df = new_var(temp_df)
        
        # concatenate the temporary dataframe to the bundle_specs_df
        bundle_specs_df = pd.concat([bundle_specs_df, temp_df], ignore_index=True)
    
    return bundle_specs_df, bundle_data_valid

bundle_specs_df, valid_bundle = create_bundle_data(bundle_data, data_train_with_dummy)
bundle_specs_df.shape

(7028, 424)

# 7. Forecasting

In [47]:
# difference between the bundle columns and the original data columns
diff = list(set(data_train_with_dummy.columns) - set(bundle_specs_df.columns))
print(diff)

['sales_units']


In [48]:
# match the columns
bundle_cols = bundle_specs_df.columns
rf_cols = x_train_promo_rf.columns

bundle_copy = bundle_specs_df.copy()
bundle_copy = bundle_copy[rf_cols]

# predict the sales units using rf (best model)
best_model = rf_promo
bundle_sales_units = best_model.predict(bundle_copy)

# add the predicted sales units to the bundle data
bundle_specs_df['sales_units'] = bundle_sales_units

In [49]:
# order the bundle data by sales units
bundle_specs_df_agg = bundle_specs_df.copy()

# aggregate the calendar days in 2 weeks
bundle_specs_df_agg['calendar_day'] = pd.to_datetime(bundle_specs_df_agg['calendar_day'])
print(bundle_specs_df_agg.shape)

# group by article_id and aggregate the sales units
bundle_specs_df_agg = bundle_specs_df_agg.groupby('article_id').agg({'sales_units': 'sum'}).reset_index()
bundle_specs_df_agg = bundle_specs_df_agg.sort_values(by='sales_units', ascending=False)

# get the description of the top 5 bundles
bundle_specs_df_agg['Top Article Description'] = [0]*bundle_specs_df_agg.shape[0]
bundle_specs_df_agg['Other Article Description'] = [0]*bundle_specs_df_agg.shape[0]

for idx in bundle_specs_df_agg.index:
    article_id = bundle_specs_df_agg['article_id'][idx]
    top_article_desc = bundle_data[bundle_data['bundle_id'] == article_id]['Top Article Description'].values[0]
    other_article_desc = bundle_data[bundle_data['bundle_id'] == article_id]['Other Article Description'].values[0]
    bundle_specs_df_agg.loc[idx, 'Top Article Description'] = top_article_desc
    bundle_specs_df_agg.loc[idx, 'Other Article Description'] = other_article_desc
    bundle_specs_df_agg.loc[idx, 'Bundle Price'] = bundle_specs_df[bundle_specs_df['article_id'] == article_id]['sell_price'].values[0]
    
bundle_specs_df_agg.head()

(7028, 425)


,article_id,sales_units,Top Article Description,Other Article Description,Bundle Price
17,430030,1174.383936,Strike Disinfectant Wipes Lemon 100pk,Fab Subblime Velvet 15kg,40.335
18,441278,1172.983115,Strike Disinfectant Wipes Lemon 100pk,TT Made Fof Me Breast Pads - M (40PK),13.015
21,505208,1108.525784,Strike Disinfectant Wipes Lemon 100pk,Dove Powder Soft Roll On 50ml,5.415
15,392445,1084.478383,Strike Disinfectant Wipes Lemon 100pk,Hug UDPant Size 3 Crawler Boy 36 Bulk,13.910
10,338104,1072.379254,Strike Disinfectant Wipes Lemon 100pk,Bondi Sands Pure Water Light/Med 200ml,18.365


# 8. Evaluate the performance of bundles
- Apply the bundle discount to the low performing articles
- Compare the performance of the bigW sales

In [50]:
# sales before price change
sales_before = data_train_with_dummy['sales_units'].sum()
print(f"Total Sales Before Bundle Applied: {sales_before} units")

Total Sales Before Bundle Applied: 26686855 units


In [51]:
# obtain top 10 bundles
top_10_bundles_id = bundle_specs_df_agg.head(10)['article_id']
top_10_bundles_id = top_10_bundles_id.tolist()
top_10_bundles_id

[430030,
 441278,
 505208,
 392445,
 338104,
 220109,
 611760,
 340350,
 500238,
 820695]

In [52]:
# mimick bundle sales by applying the total price change on the low performing article
bundle_impact = bundle_specs_df.copy()
data_train_with_dummy_after_bundle = data_train_with_dummy.copy()
best_model = rf_promo

# select the top 10 bundles
bundle_impact = bundle_impact[bundle_impact['article_id'].isin(top_10_bundles_id)]

# get the sell price of the top and low performing articles on that day
bundle_impact['top_sell_price'] = [0]*bundle_impact.shape[0]
bundle_impact['other_sell_price'] = [0]*bundle_impact.shape[0]

for idx in bundle_impact.index:
    # find the bundle id for top and low performing articles
    bundle_id = bundle_impact['article_id'][idx]
    top_article_id = valid_bundle[valid_bundle['bundle_id'] == bundle_id]['Top Article ID'].values[0]
    other_article_id = valid_bundle[valid_bundle['bundle_id'] == bundle_id]['Other Article ID'].values[0]
    
    # find the bundle sell price
    bundle_sell_price = bundle_impact['sell_price'][idx]
    
    # find the sell price of the top and low performing articles on that day
    day = bundle_impact['calendar_day'][idx]
    top_sell_price = data_train_with_dummy_after_bundle[(data_train_with_dummy_after_bundle['article_id'] == top_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day)]['sell_price'].values[0]
    low_sell_price = data_train_with_dummy_after_bundle[(data_train_with_dummy_after_bundle['article_id'] == other_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day)]['sell_price'].values[0]
    
    # sum the sell price of the top and low performing articles
    sum_sell_price = top_sell_price + low_sell_price
    
    # calculate the price change
    price_change = sum_sell_price - bundle_sell_price
    
    # apply the price change to the original dataset
    data_train_with_dummy_after_bundle.loc[(data_train_with_dummy_after_bundle['article_id'] == other_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day), 'sell_price'] = low_sell_price - price_change
    
    # find the rows that have the same calendar day and article id
    row = data_train_with_dummy_after_bundle[(data_train_with_dummy_after_bundle['article_id'] == other_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day)]
    
    # use the best model to predict the sales units given the price change
    new_sales_units = best_model.predict(row[rf_cols])
    
    # update the gross profit = sales units * sell price - cost of purchasing + scanback
    new_profit = new_sales_units * row['sell_price'] - row['cost_of_purchasing'] + row['scanback']
    data_train_with_dummy_after_bundle.loc[(data_train_with_dummy_after_bundle['article_id'] == other_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day), 'net_profit'] = new_profit
    
    # update the sales units
    data_train_with_dummy_after_bundle.loc[(data_train_with_dummy_after_bundle['article_id'] == other_article_id) & (data_train_with_dummy_after_bundle['calendar_day'] == day), 'sales_units'] = new_sales_units
    
# sales after price change
sales_after = data_train_with_dummy_after_bundle['sales_units'].sum()
sales_after

26689946.21945626

In [53]:
# calculate the increase in sales after apply 10 bundles
sales_increase = round(sales_after - sales_before)
print(f"Sales Increase: {sales_increase} units")

percentage_increase = (sales_increase / sales_before) * 100
print(f"Percentage Increase: {percentage_increase}%")

Sales Increase: 3091 units
Percentage Increase: 0.011582481337722262%


# 9. Checking Gross Profit

In [54]:
# profit before price change
profit_before = data_train_with_dummy['net_profit'].sum()
print("Total Net Profit Before Bundle Applied: ${:.2f}".format(profit_before))

profit_after = data_train_with_dummy_after_bundle['net_profit'].sum()
print("Total Net Profit After Bundle Applied: ${:.2f}".format(profit_after))

profit_increase = round(profit_after - profit_before)
print(f"Net Profit Increase: ${profit_increase}")

Total Net Profit Before Bundle Applied: $76394829.54
Total Net Profit After Bundle Applied: $76418660.11
Net Profit Increase: $23831


# 10. Other Metrics for the Best Model

In [55]:
# get the regression report on the model
def regression_report(model, X_data_unstd, y_data_unstd):
    # evaluate the model
    y_pred = model.predict(X_data_unstd)
    
    # rmse, mse, mae
    rmse = np.sqrt(mean_squared_error(y_data_unstd, y_pred))
    mse = mean_squared_error(y_data_unstd, y_pred)
    mae = mean_absolute_error(y_data_unstd, y_pred)
    
    return rmse, mse, mae
    

In [56]:
# get the best model and evaluate on the unstandardized whole data
best_model = rf_promo

y_train, x_train = y_x_split(data_train_with_dummy)
y_val, x_val = y_x_split(data_val_with_dummy)
y_test, x_test = y_x_split(data_test_with_dummy)

# concatenate the training, validation and test data
y_data_unstd = pd.concat([y_train, y_val, y_test], ignore_index=True)
x_data_unstd = pd.concat([x_train, x_val, x_test], ignore_index=True)

# get the fitted features for X
x_data_unstd = x_data_unstd[x_train_promo_rf.columns]

# get the regression report
rmse, mse, mae = regression_report(best_model, x_data_unstd, y_data_unstd)

pd.DataFrame({'Model': ['RF for Promo Sales'], 
                    'Mean Squared Error': [mse], 
                    'Mean Absolute Error': [mae],
                    'Root Mean Squared Error': [rmse]})


,Model,Mean Squared Error,Mean Absolute Error,Root Mean Squared Error
0,RF for Promo Sales,3982.585398,21.062069,63.107729
